### Deps

In [ ]:
import re
import requests
import random
import time
import pandas as pd
from bs4 import BeautifulSoup

### Extração dos links

In [ ]:
def sleep_random(min_ms=10, max_ms=2000):
    tempo_ms = random.randint(min_ms, max_ms)
    time.sleep(tempo_ms / 1000)

In [ ]:
def get_soup_nav(page) -> BeautifulSoup:
        url = "https://mcreator.net/modifications?page={}#google_vignette".format(page)
        response = requests.get(url)

        if response.status_code == 200:
            return BeautifulSoup(response.text, 'html.parser')
        
        print("Failed to retrieve page {}: {}".format(page, response.status_code))
        return None

In [ ]:
def get_links(soup: BeautifulSoup) -> list:
    links = []
    for link in soup.find_all('a'):
        href = link.get('href')
        if href:
            links.append(href)
    return links

In [ ]:
def get_all_links() -> list:
    page = 1
    all_links = []

    while True:
        sleep_random()

        soup = get_soup_nav(page)
        if soup is None:
            break

        links = get_links(soup)

        pattern = re.compile(r'.*modification/\d+/.*', re.IGNORECASE)
        filtered_links = [link for link in links if pattern.match(link)]

        if not filtered_links:
            print("No more links found on page {}".format(page))
            continue
        else:
            print("Found {} links on page {}".format(len(filtered_links), page))
        
        all_links.extend(filtered_links)

        if page == 2:
            print("Stopping after 2 pages.")
            break

        page += 1
    return all_links

### Extração dos meta-dados

In [ ]:
def get_soup_mod(link) -> BeautifulSoup:
    url = "https://mcreator.net" + link
    response = requests.get(url)
    if response.status_code == 200:
        return BeautifulSoup(response.text, 'html.parser')
    
    print("Failed to retrieve mod page: {}, link {}".format(response.status_code, link))
    return None

In [ ]:
def get_name(soup: BeautifulSoup) -> str:
    h1_tag = soup.find('h1')
    return h1_tag.text.strip() if h1_tag else None

In [ ]:
def get_description(soup: BeautifulSoup) -> str:
    pass

In [ ]:
def get_author(soup: BeautifulSoup) -> str:
    pass

In [ ]:
def get_date_published(soup: BeautifulSoup) -> str:
    pass

In [ ]:
def get_date_updated(soup: BeautifulSoup) -> str:
    pass

In [ ]:
def get_downloads(soup: BeautifulSoup) -> int:
    pass

In [ ]:
def get_views(soup: BeautifulSoup) -> int:
    pass

In [ ]:
def get_version(soup: BeautifulSoup) -> str:
    pass

In [ ]:
def get_category(soup: BeautifulSoup) -> str:
    pass


In [ ]:
def get_tags(soup: BeautifulSoup) -> list:
    pass

In [ ]:
def get_modloader(soup: BeautifulSoup) -> str:
    pass

In [ ]:
def get_amount_updates(soup: BeautifulSoup) -> int:
    pass

In [ ]:
def get_size(soup: BeautifulSoup) -> float:
    pass


In [ ]:
def get_dependencies(soup: BeautifulSoup) -> list:
    pass

In [ ]:
def get_links(soup: BeautifulSoup) -> list:
    pass


In [ ]:
def get_source(soup: BeautifulSoup) -> str:
    pass

In [ ]:
def get_data_mod(soup: BeautifulSoup) -> dict:
    data = {'name': '',
            'description': '',
            'author': '',
            'date_published': '',
            'date_updated': '',
            'downloads': 0,
            'views': 0,
            'version': '',
            'category': '',
            'tags': [],
            'modloader': '',
            'amount_updates': 0,
            'size': 0,
            'dependencies': [],
            'link': [],
            'source':''}
    
    data['name'] = get_name(soup)
    data['description'] = get_description(soup)
    data['author'] = get_author(soup)
    data['date_published'] = get_date_published(soup)
    data['date_updated'] = get_date_updated(soup)
    data['downloads'] = get_downloads(soup)
    data['views'] = get_views(soup)
    data['version'] = get_version(soup)
    data['category'] = get_category(soup)
    data['tags'] = get_tags(soup)
    data['modloader'] = get_modloader(soup)
    data['amount_updates'] = get_amount_updates(soup)
    data['size'] = get_size(soup)
    data['dependencies'] = get_dependencies(soup)
    data['link'] = get_links(soup)
    data['source'] = get_source(soup)

    return data


In [ ]:
def get_data_mcreator_net(links) -> pd.DataFrame:
    mods = []
    for link in links:
        sleep_random(0, 500)
        soup = get_soup_mod(link)
        if soup is None:
            continue

        data = get_data_mod(soup)
        if data is None:
            continue
        mods.append(data)
    
    df = pd.DataFrame(mods)
    return df

In [ ]:
links = get_all_links()

In [ ]:
df = get_data_mcreator_net(links)